In [10]:
from binance.client import Client
from datetime import datetime, timezone
import csv

# Claves API de tu cuenta mock
api_key = '7de4ca57d53aac719f84f4f2f1ceee269512766e19734dca3beaec464c92a2d1'
api_secret = '15122b41c53e6293bd122333bf8f31a12e7f8b876595a290473487e38a25d3f0'

# Conexión con Binance Futures testnet
client = Client(api_key, api_secret, testnet=True)
client.FUTURES_URL = "https://testnet.binancefuture.com/fapi/v1"

In [11]:
def list_available_pairs():
    """List all available trading pairs on Binance Futures."""
    exchange_info = client.futures_exchange_info()
    symbols = [symbol['symbol'] for symbol in exchange_info['symbols']]
    return symbols

# Ejecuta para listar pares
available_pairs = list_available_pairs()

print(f"Available pairs: {available_pairs}")


Available pairs: ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'DASHUSDT', 'ZECUSDT', 'XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT', 'VETUSDT', 'NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT', 'ZRXUSDT', 'COMPUSDT', 'OMGUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT', 'RLCUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT', 'BALUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'EGLDUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'ENJUSDT', 'FLMUSDT', 'TOMOUSDT', 'RENUSDT', 'KSMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'OCEANUSDT', 'BELUSDT', 'CTKUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT', 'BNTUSDT', '1INCHUSDT', 'UNFIUSDT', 'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'LITUSDT', 'REEFUSDT', 'RVNUSDT', 'SFPUSDT', 'XEMUSDT', 'BTCSTUSDT', 'COTIUSDT', 'CHRUSDT', 'MANAUSD

In [12]:
def get_current_prices(pairs):
    """Fetch the current prices for the specified trading pairs."""
    prices = {}
    for pair in pairs:
        try:
            ticker = client.futures_symbol_ticker(symbol=pair)
            prices[pair] = float(ticker['price'])
        except Exception as e:
            print(f"Error fetching price for {pair}: {e}")
    return prices

# Especifica los pares
pairs_to_check = ["BTCUSDT", "ETHUSDT", "BNBUSDT"]

# Ejecuta para obtener precios actuales
current_prices = get_current_prices(pairs_to_check)

print("Current prices:")

for pair, price in current_prices.items():
    print(f"{pair}: {price}")


Current prices:
BTCUSDT: 97538.8
ETHUSDT: 3243.22
BNBUSDT: 702.74


In [13]:
def get_historical_prices(symbol, interval, start_time=None, end_time=None, limit=100):
    """
    Fetch historical price data (candlesticks) for a given trading pair.
    """
    try:
        if isinstance(start_time, str):
            start_time = int(datetime.strptime(start_time, "%Y-%m-%d").timestamp() * 1000)
        if isinstance(end_time, str):
            end_time = int(datetime.strptime(end_time, "%Y-%m-%d").timestamp() * 1000)

        klines = client.futures_klines(
            symbol=symbol,
            interval=interval,
            startTime=start_time,
            endTime=end_time,
            limit=limit
        )
        return klines
    except Exception as e:
        print(f"Error fetching historical prices for {symbol}: {e}")
        return []

# Obtén datos históricos de BTC
btc_historical = get_historical_prices("BTCUSDT", "5m", limit=500)
btc_historical[:5]  # Muestra las primeras 5 filas


[[1736815500000,
  '93619.60',
  '94600.00',
  '92241.50',
  '93212.70',
  '11.051',
  1736815799999,
  '1031583.01600',
  222,
  '5.579',
  '524346.06030',
  '0'],
 [1736815800000,
  '93060.00',
  '93500.00',
  '92241.50',
  '92986.40',
  '5.712',
  1736816099999,
  '529907.74810',
  158,
  '2.220',
  '207273.22260',
  '0'],
 [1736816100000,
  '92998.00',
  '93499.00',
  '92241.50',
  '93134.30',
  '4.032',
  1736816399999,
  '374278.28500',
  92,
  '1.926',
  '179901.37250',
  '0'],
 [1736816400000,
  '93134.30',
  '93499.00',
  '92800.00',
  '93000.00',
  '3.614',
  1736816699999,
  '336323.04010',
  66,
  '1.016',
  '94958.86960',
  '0'],
 [1736816700000,
  '93297.50',
  '94650.00',
  '92801.00',
  '94149.80',
  '40.284',
  1736816999999,
  '3762590.39420',
  278,
  '13.592',
  '1280949.25190',
  '0']]

In [15]:
import pandas as pd
import plotly.graph_objects as go

def plot_chart(btc_historical, chart_type="line"):
    # Asegúrate de que los valores de tiempo están en milisegundos
    btc_historical_corrected = [
        [kline[0], *kline[1:6], kline[6], *kline[7:]] for kline in btc_historical
    ]

    # Convierte datos a DataFrame
    df = pd.DataFrame(
        btc_historical_corrected,
        columns=["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "Quote Asset Volume", "Number of Trades", "Taker Buy Base Volume", "Taker Buy Quote Volume", "Ignore"]
    )

    # Convertir el tiempo a formato datetime
    df["Open Time"] = pd.to_datetime(df["Open Time"], unit="ms")
    df["Open"] = df["Open"].astype(float)
    df["High"] = df["High"].astype(float)
    df["Low"] = df["Low"].astype(float)
    df["Close"] = df["Close"].astype(float)

    # Verifica los primeros registros
    print(df.head())

    if chart_type == "line":
        # Gráfico de líneas con plotly
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df["Open Time"], y=df["Close"], mode='lines', name='Precio de cierre'))

        fig.update_layout(
            title="BTCUSDT - Precios de cierre",
            xaxis_title="Tiempo",
            yaxis_title="Precio",
            xaxis_rangeslider_visible=False,  # Desactivar el slider de rango
            dragmode='zoom',  # Habilitar desplazamiento y zoom con el mouse
            hovermode='x unified',  # Mostrar todos los valores en un punto a la vez
            template="plotly_dark",  # Estilo oscuro opcional
            width=1200,  # Ajustar el ancho del gráfico
            height=600,  # Ajustar el alto del gráfico
            autosize=True,  # Hacer el gráfico más adaptable
            xaxis=dict(
                rangeslider=dict(visible=False),  # No mostrar el slider
                showgrid=False  # Opcional, para un fondo más limpio
            ),
            yaxis=dict(showgrid=True)  # Mostrar cuadrícula para los valores del eje Y
        )
        fig.show()

    elif chart_type == "candle":
        # Gráfico de velas con plotly
        fig = go.Figure(data=[go.Candlestick(
            x=df["Open Time"],
            open=df["Open"],
            high=df["High"],
            low=df["Low"],
            close=df["Close"],
            increasing_line_color="green", 
            decreasing_line_color="red"
        )])

        fig.update_layout(
            title="BTCUSDT - Gráfico de Velas",
            xaxis_title="Tiempo",
            yaxis_title="Precio",
            xaxis_rangeslider_visible=False,  # Desactivar el slider de rango
            dragmode='zoom',  # Habilitar desplazamiento y zoom
            hovermode='x unified',  # Mostrar todos los valores en un punto a la vez
            template="plotly_dark",  # Estilo oscuro opcional
            width=1200,  # Ajustar el ancho del gráfico
            height=600,  # Ajustar el alto del gráfico
            autosize=True,  # Hacer el gráfico más adaptable
            xaxis=dict(
                rangeslider=dict(visible=False),  # No mostrar el slider
                showgrid=False  # Opcional, para un fondo más limpio
            ),
            yaxis=dict(showgrid=True)  # Mostrar cuadrícula para los valores del eje Y
        )
        fig.show()

    else:
        raise ValueError("chart_type debe ser 'line' o 'candle'.")

# Llamar la función para probar
plot_chart(btc_historical, chart_type="candle")


            Open Time     Open     High      Low    Close  Volume  \
0 2025-01-14 00:45:00  93619.6  94600.0  92241.5  93212.7  11.051   
1 2025-01-14 00:50:00  93060.0  93500.0  92241.5  92986.4   5.712   
2 2025-01-14 00:55:00  92998.0  93499.0  92241.5  93134.3   4.032   
3 2025-01-14 01:00:00  93134.3  93499.0  92800.0  93000.0   3.614   
4 2025-01-14 01:05:00  93297.5  94650.0  92801.0  94149.8  40.284   

      Close Time Quote Asset Volume  Number of Trades Taker Buy Base Volume  \
0  1736815799999      1031583.01600               222                 5.579   
1  1736816099999       529907.74810               158                 2.220   
2  1736816399999       374278.28500                92                 1.926   
3  1736816699999       336323.04010                66                 1.016   
4  1736816999999      3762590.39420               278                13.592   

  Taker Buy Quote Volume Ignore  
0           524346.06030      0  
1           207273.22260      0  
2       

In [16]:
def export_to_csv(klines, symbol, interval):
    """
    Export kline (candlestick) data to a CSV file with a dynamic filename.
    
    The filename includes the symbol, interval, and current date to avoid overwriting.
    """
    # Generar el nombre del archivo dinámicamente
    current_date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')  # Fecha y hora actual
    filename = f"data/{symbol}_{interval}_{current_date}.csv"
    
    headers = [
        "Open Time", "Open", "High", "Low", "Close", "Volume",
        "Close Time", "Quote Asset Volume", "Number of Trades",
        "Taker Buy Base Volume", "Taker Buy Quote Volume", "Ignore"
    ]
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write the header row
        for kline in klines:
            kline[0] = datetime.fromtimestamp(kline[0] / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
            kline[6] = datetime.fromtimestamp(kline[6] / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
            writer.writerow(kline)  # Write each kline as a row
    
    print(f"Historical data exported to {filename}")

# Exporta los datos históricos de BTC a un archivo CSV
export_to_csv(btc_historical, "BTCUSDT", "5m")


Historical data exported to data/BTCUSDT_5m_2025-01-15_13-33-40.csv
